<h1 align="center">Markov Chain Monte Carlo for fun and profit</h1>
<h1 align="center"> 🎲 ⛓️ 👉 🧪 </h1>

# Testing

Ok we can finally start writing and running some tests! Check out the [pytest website](https://docs.pytest.org/en/7.1.x/getting-started.html) for a tutorial on how to write tests in pytest and head over to the [Turing Way](https://the-turing-way.netlify.app/reproducible-research/testing.html) for a great introduction to testing in general. 

I copied some of the initial tests that we did in chapter 1 into `test_energy.py` installed pytest into my development environment with `pip install pytest`. If you're using conda you need to use `conda install pytest` and now I can run the `pytest` command in the ReCoDE_MCFF directory. Pytest will automatically discover our tests and run them, to do this it relies on their being python files with functions named `test_\*` which it will run.

If that doesn't work and complains it can't find MCFF, try `python -m pytest`, this asks python to find a module and run it, which can be useful to ensure you're running pytest inside the correct environment. I ran into this problem because I used `pip install pytest` into a conda environment when I should have done `conda install pytest`.

But hopefully you can get it working and get a lovely testy output! I've embedded a little video of this below but if it doesn't load, check out the [link](https://asciinema.org/a/498583)

In [1]:
%%html 
<div style="max-width:700px;margin:auto;"><script id="asciicast-498583" src="https://asciinema.org/a/498583.js" async></script></div>

## Basic Testing with Pytest

Take a look at `tests/test_energy.py`. You can see that I've done some imports, setup some test states and then defined two testing functions:

```python
# Note that only functions whose name begins with test_ get run by pytest
def E_prediction_all_the_same(L): 
    "The exact energy in for the case where all spins are up or down"
    return -(4*(L - 2)**2 + 12*(L-2) + 8)

def test_exact_energies():
    for state in [all_up, all_down]:
        L = state.shape[0]
        assert energy(state) == E_prediction_all_the_same(L)
```

I will defer to external resources for a full discussion of the philosphy of testing but I generally think of tests as an aid to my future debugging. If I make a change that breaks something then I want my tests to catch that and to make it clear what has broken. As such I generally put tests that check very basic properties of my code early on in the file and then follow them with tests that probe more subtle things or more obscure edges cases.

`test_exact_energies` checks that the energies of our exact states come out as we calculated they should in chapter 1. This is testing a very limited space of the possible inputs to `energy` so we'd like to find some way to be more confident that our implementation is correct.

One was is to test multiple independant implementations against one another: `test_energy_implementations` checks our numpy implementation against our numba one. This should catch implementation bugs because it's unlikely we will make the same such error in both implementations. 

```python
def test_energy_implementations():
    for state in states:
        assert np.allclose(energy(state), energy_numpy(state))
```

However if we have made some logical errors in how we've defined the energy, that error will likely appear in both implememtations and thus won't be caught by this. 

Generally what we will do now, is that as we write more code or add new functionality we will add tests to check that functionality.

## Coverage Testing

A useful little trick for testing, are tools like pytest-cov that can measure *coverage*, that is, the amount of your code base that is activate by your tests. Unfortunatley Numba does not play super well with pytest-cov so we have to turn off numba to generate the test report using an environment variable.

```bash
(recode) tom@TomsLaptop ReCoDE_MCMCFF % pip install pytest-cov # install the coverage testing plugin
(recode) tom@TomsLaptop ReCoDE_MCMCFF % NUMBA_DISABLE_JIT=1 pytest --cov=MCFF --cov-report=term

================================================== test session starts ==================================================
platform darwin -- Python 3.9.12, pytest-7.1.1, pluggy-1.0.0
rootdir: /Users/tom/git/ReCoDE_MCMCFF
plugins: hypothesis-6.46.10, cov-3.0.0
collected 3 items                                                                                                       

code/tests/test_energy.py ..                                                                                      [ 66%]
code/tests/test_energy_using_hypothesis.py .                                                                      [100%]

---------- coverage: platform darwin, python 3.9.12-final-0 ----------
Name                           Stmts   Miss  Cover
--------------------------------------------------
code/src/MCFF/__init__.py          0      0   100%
code/src/MCFF/ising_model.py      22      3    86%
code/src/MCFF/mcmc.py             14     14     0%
--------------------------------------------------
TOTAL                             36     17    53%


=================================================== 3 passed in 1.89s ===================================================
```

Ok so this is telling us that we currently test 86% of the lines in ising_model.py. We can also change `--cov-report=html` to get a really nice html output which shows which parts of your code aren't being run.

A warning though, testing 100% of your lines of code doesn't mean it's correct, you need to think carefully about the data you test on, try to pick the hardest examples you can think of! What edge cases might there be that would break your code? Zero, empty strings and empty arrays are classic examples.

## Advanced Testing Methods: Hypothesis


I won't do into huge detail here but I thought it would be nice to make you aware of a nice library called `Hypothesis` that helps with this problem of finding edge cases. `Hypothesis` gives you tools to generate randomised inputs to functions, so as long as you can come up with some way to verify the output is correct (or just that the code doens't throw and error!) then this can be a powerful method of testing. 


Take a look in `test_energy_using_hypothesis.py`
```python
from hypothesis.extra import numpy as hnp

@given(hnp.arrays(dtype = int,
                 shape = hnp.array_shapes(min_dims = 2, max_dims = 2),
                 elements = st.sampled_from([1, -1])))
def test_generated_states(state):
    assert np.allclose(energy(state), energy_numpy(state))
```
You tell Hypothesis how to generate the test data, in this case we use some numpy specifc code to generate 2 dimensional arrays with `dtype = int` and entries randomly sampled from `[1, -1]`. We use the same trick as before of checking two implementations against one another.

## Autoformaters
Resources: [](https://the-turing-way.netlify.app/reproducible-research/code-quality/code-quality-style.html)

While we're doing things that will help keep our code clean and tidy in the future, I would recommend installing a code formatter like `black`. This is a program that enforces a particular formatting style on your code by simply doing it for you. At first this sounds a bit weird, but it has a few benefits:

- It makes git diffs as small as possible because formatting changes never show up
- It means you never have to discuss with your collaborators about code formatting, something which can waste a lot of time!

Here I will show you how to setup `black` as a pre-commit hook, this means it runs before you commit anything to git, which is probably the best time to do it. We'll use a helper tool called [pre-commit](https://pre-commit.com/).

```bash
pip install pre-commit
pre-commit sample-config >> .pre-commit-config.yaml # Generate an initial config
```
Now we add some additional lines to the `.pre-commit-config.yaml` config file to setup black:
```yaml
-   repo: https://github.com/psf/black
    rev: 21.12b0
    hooks:
    -   id: black
    -   id: black-jupyter
```
And finally `pre-commit install` will make this run every time you commit to git. It's worth running it manually once the first time to check it works: `pre-commit run --all-files`. Running this I immediatly got a cryptic error that, on googling, turned out to be that something broke in version 21.12b0 of `21.12b0`. Running `precommit autoupdate` fixed this for me by updated `black` to a later version. Running `pre-commit run --all-files` a second time now gives me:
```bash
(recode) tom@TomsLaptop ReCoDE_MCMCFF % pre-commit run --all-files
trim trailing whitespace.................................................Passed
fix end of files.........................................................Passed
check yaml...........................................(no files to check)Skipped
check for added large files..............................................Passed
black....................................................................Passed
(recode) tom@TomsLaptop ReCoDE_MCMCFF % 
```

Now whenever you commit anything, `black` will autoformat it before it actually gets commited. I'll test this for myself by putting
```python
def ugly_litte_one_liner(a,b,c): return " ".join([a,b,c/2. +3])
```
in a code cell below and we'll see how `black` formats it. The only gotcha here is that you have to reload jupyter notebooks from disk in order to see the changes that `black` makes.

In [2]:
def ugly_litte_one_liner(a, b, c):
    return " ".join([a, b, c / 2.0 + 3])

Finally, be aware that if you try to commit code with incorrect syntax then black will just error and prevent it, this is probably a good thing but there may be the occasional time where that's a problem.

## Testing Stochastic Code



...

We can use a t-test to check a sample from the MCMC sampler matches our expectations. 

In [4]:
# observations = average_color_data.mean(axis = -1)

# from scipy import stats
# stats.ttest_1samp(observations[0], 0).pvalue < 1 / 50

## Test Driven Development

